In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

datestr = '20250901'

# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')


# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")),
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1) #找到「證券代號」那一行作為表頭，並讀成乾淨的 DataFrame

# 整理一些字串：
# errors='coerce' 表示把不能轉換的值 → 變成 NaN
df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))

df[
    (pd.to_numeric(df['本益比'], errors='coerce') > 0) &
    (pd.to_numeric(df['本益比'], errors='coerce') < 15)
]

# 本益比（P/E Ratio, Price-to-Earnings Ratio）是股票投資裡最常見的估值指標之一。
# 公式是：本益比 = 股價 (Price)/ 每股盈餘 (EPS, Earnings Per Share)
# --------------------------------------------------------------------------
# 股價：你要花多少錢買一股。
# EPS：公司一股能賺多少錢（淨利 ÷ 股數）。
# 👉本益比 =「投資人願意花幾倍的價格，去買公司 1 元的獲利」。
# ---------------------------------------------------------------------------
# 舉例：假設台積電股價 = 600 元，EPS = 30 元
# 本益比 = 600 ÷ 30 = 20 (也就是說，投資人願意花 20倍的錢 去買公司一年賺的利潤)
# ---------------------------------------------------------------------------
# 投資上的意義:
# 高本益比（例如 40、50 以上）→ 表示市場看好，覺得公司未來會有高成長，但也代表股價偏貴。
# 低本益比（例如 10 以下）→ 可能被低估，或市場覺得成長有限，甚至是公司有風險。